In [1]:
import torch
import numpy as np

# Conjugate gradient algorithms

We want to find a solution to the linear system
$$
Ax=b
$$
by solving the following quadratic program:
$$
\min_{x} \| Ax - b \|^{2}
$$

The following functions implement a conjugate gradient, one using Numpy, the other one using Pytorch.

In [2]:
def cg_numpy(
    A,
    b,
    eps_a=10**-14,
    eps_r=10**-14,
    nIter_max=1000
):
    """
    Solve Ax=b with conjugate gradient on normal equations.
    """
    
    m, n = A.shape
    
    # initial point and residuals
    x = np.zeros(n)
    r = -A.T.dot(b)

    p = -r
    r_0_norm = np.linalg.norm(r)
    beta = 0.0

    keep_going = True
    iter_count = 0

    for k in range(1, nIter_max+1):

        r_sqnorm = r.dot(r)
        keep_going = (np.sqrt(r_sqnorm) > eps_a+eps_r*r_0_norm)

        # keep updating while stopping criterion is not met
        if(keep_going):
            iter_count += 1
            
            # storing q speeds up computations
            q = A.dot(p) 

            # compute step size
            alpha = r_sqnorm / q.T.dot(q)

            # perform update on x
            x += alpha*p

            # compute new conjugate gradient
            r +=  alpha*A.T.dot(q)

            beta =  r.dot(r)/ r_sqnorm
            p =  - r+beta*p
        else:
            break
            
    print('{} iterations'.format(iter_count))

    return x

In [3]:
def cg_torch(
    A,
    b,
    eps_a=10**-14,
    eps_r=10**-14,
    nIter_max=1000
):
    """
    Solve Ax=b with conjugate gradient on normal equations.
    """
    
    m, n = A.size()
    
    # initial point and residuals
    x = torch.DoubleTensor(n).zero_().cuda()
    r = -torch.matmul(torch.t(A), b)

    p = -r
    r_0_norm = torch.norm(r)
    beta = 0.0

    keep_going = True
    iter_count = 0

    for k in range(1, nIter_max+1):

        r_sqnorm = torch.matmul(r, r)
        keep_going = (np.sqrt(r_sqnorm) > eps_a+eps_r*r_0_norm)

        # keep updating while stopping criterion is not met
        if(keep_going):
            iter_count += 1
            
            # storing q speeds up computations
            q = torch.matmul(A, p)

            # compute step size
            alpha = r_sqnorm / (torch.matmul(q, q))

            # perform update on x
            x += alpha * p

            # compute new conjugate gradient
            r +=  alpha * torch.matmul(torch.t(A), q)

            beta =  torch.matmul(r, r) / r_sqnorm
            p =  -r + beta * p
        else:
            break
            
    print('{} iterations'.format(iter_count))

    return x

# Numerical tests

Compare the performance of both methods on numerical examples

In [4]:
import time

In [5]:
# define dimensions
m, n = 1000, 1000

# create matrix and right hand side
A = np.random.rand(m, n)
b = np.ones(m)

A_ = torch.DoubleTensor(A).cuda()
b_ = torch.DoubleTensor(b).cuda()

In [6]:
start = time.clock()
x_ = cg_torch(A_, b_)
end = time.clock()

print((end-start))

1000 iterations
0.2825129999999998


In [7]:
start = time.clock()
x = cg_numpy(A, b)
end = time.clock()

print((end-start))

1000 iterations
1.8259469999999998
